In [3]:
from bs4 import BeautifulSoup
from newsapi import NewsApiClient
import pandas as pd
from pytrends.request import TrendReq
import requests
import yfinance as yf

In [4]:
## Dict of dataframes created along the notebook
dfs = {}

## Web Scraping from Wikipedia - Companies in the DAX as of 22 September 2025

In [5]:
url = "https://de.wikipedia.org/wiki/DAX"
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}
response = requests.get(url, headers=headers)
print(response)
soup = BeautifulSoup(response.content, "html.parser")

<Response [200]>


In [6]:
companies = []
tickers = []
industries = []
for ind, element in enumerate(soup.select("tbody tr td")[21:340]):
    if ind % 8 == 0:
        companies.append(element.get_text().strip())
    elif ind % 8 == 1:
        tickers.append(element.get_text())
    elif ind % 8 == 2:
        industries.append(element.get_text())

In [7]:
DAX = pd.DataFrame({
    "company":companies,
    "ticker":tickers,
    "industry":industries
})

In [8]:
# Add termination to tickers so they are readable by Yahoo Finance API

exception = {
    "Airbus": "AIR.PA"   # only non-DE listing
}

DAX["ticker_yahoo"] = DAX["ticker"].apply(lambda t: t + ".DE")

# Apply only real exceptions
DAX["ticker_yahoo"] = DAX.apply(
    lambda x: exception.get(x["company"], x["ticker_yahoo"]),
    axis=1
)

In [24]:
translations = {
    "Sportartikel": "Sporting goods",
    "Luftfahrt, Raumfahrt, Rüstung": "Aerospace",
    "Luftfahrt": "Aerospace",
    "Versicherungen": "Insurance",
    "Chemie": "Chemicals & Pharmaceuticals",
    "Chemie, Pharma": "Chemicals & Pharmaceuticals",
    "Konsumgüter (Produktion)": "Consumer goods",
    "Automobil (Produktion)": "Automotive industry",
    "Chemie (Handel)": "Chemicals & Pharmaceuticals",
    "Banken": "Banks",
    "Automobil (Zulieferer)": "Automotive industry",
    "Nutzfahrzeuge (Produktion)": "Automotive industry",
    "Börsen": "Stock exchange",
    "Logistik": "Logistics",
    "Telekommunikation": "Telecommunications",
    "Energie (Versorgung)": "Energy",
    "Medizintechnik, Kliniken": "Medical technology",
    "Medizintechnik": "Hospitals",
    "Maschinenbau": "Mechanical engineering",
    "Baustoffe": "Building materials",
    "Konsumgüter, Chemie": "Consumer goods",
    "Halbleiter": "Semiconductors",
    "Holding": "Holding company",
    "Biotechnologie": "Biotechnology",
    "Wehrtechnik, Automobil (Zulieferer)": "Defense technology",
    "Software": "Software",
    "Online-Marktplätze": "Online marketplaces",
    "Elektrotechnik": "Electrical engineering",
    "Elektrotechnik, Energietechnik": "Energy technology",
    "Lebensmittel, Chemie, Kosmetik, Konsumgüter, Duftstoffe, Aromen": "Consumer goods",
    "Immobilien": "Real estate",
    "Versandhandel (Mode)": "Fashion",
}

DAX["industry_en"] = DAX["industry"].map(translations)

In [27]:
dfs['DAX']= DAX
DAX

,company,ticker,industry,ticker_yahoo,industry_en
0,Adidas,ADS,Sportartikel,ADS.DE,Sporting goods
1,Airbus,AIR,"Luftfahrt, Raumfahrt, Rüstung",AIR.PA,Aerospace
2,Allianz,ALV,Versicherungen,ALV.DE,Insurance
3,BASF,BAS,Chemie,BAS.DE,Chemicals & Pharmaceuticals
4,Bayer,BAYN,"Chemie, Pharma",BAYN.DE,Chemicals & Pharmaceuticals
5,Beiersdorf,BEI,Konsumgüter (Produktion),BEI.DE,Consumer goods
6,BMW,BMW,Automobil (Produktion),BMW.DE,Automotive industry
7,Brenntag,BNR,Chemie (Handel),BNR.DE,Chemicals & Pharmaceuticals
8,Commerzbank,CBK,Banken,CBK.DE,Banks
9,Continental,CON,Automobil (Zulieferer),CON.DE,Automotive industry


## Yahoo! Finance's API - Import close prices of DAX companies for the last 2 years with yfinance library

In [8]:
tickers_yahoo = list(DAX['ticker_yahoo'])

# Pull data for 2 years
data = yf.download(tickers_yahoo, start="2023-10-01", end="2025-09-30", interval="1d", group_by='ticker')

/var/folders/3j/f7d6p9t136x34vdmdy30wrrc0000gn/T/ipykernel_3639/3044946893.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers_yahoo, start="2023-10-01", end="2025-09-30", interval="1d", group_by='ticker')
[*********************100%***********************]  40 of 40 completed


In [9]:
close_prices = pd.DataFrame({t: data[t]['Close'] for t in tickers_yahoo})
close_prices.dropna(inplace=True)
print(close_prices.shape)

dfs['close_prices'] = close_prices

close_prices.head()

(505, 40)


,ADS.DE,AIR.PA,ALV.DE,BAS.DE,BAYN.DE,BEI.DE,BMW.DE,BNR.DE,CBK.DE,CON.DE,...,RWE.DE,SAP.DE,G24.DE,SIE.DE,ENR.DE,SHL.DE,SY1.DE,VOW3.DE,VNA.DE,ZAL.DE
Date,,,,,,,,,,,,,,,,,,,,,
2023-10-02,162.601059,120.697922,202.678970,37.115086,44.471085,120.034706,85.925385,68.103165,9.991590,62.552277,...,31.951027,120.238358,62.604763,127.729797,12.135,46.397182,87.446121,94.590225,21.191998,21.610001
2023-10-03,158.768173,119.505936,201.086288,36.999901,43.494728,119.099625,85.148071,67.336906,9.725528,61.324299,...,30.721416,119.690308,62.254692,126.456688,11.735,46.078667,87.172310,92.941399,20.504789,20.459999
2023-10-04,160.151184,118.756142,199.903168,37.079643,43.876347,118.262978,85.237411,66.346390,9.644759,60.877766,...,30.787123,119.396706,63.129879,126.874718,11.605,46.735001,88.893456,93.340584,20.439783,20.190001
2023-10-05,154.263565,118.736916,200.858765,36.632206,43.484818,119.247269,84.245659,66.533272,9.692270,60.542862,...,31.012396,119.533722,63.052082,125.658600,11.575,46.078667,89.304192,92.420723,20.384066,20.459999
2023-10-06,157.306152,119.467491,204.135132,37.053062,43.688019,120.625290,85.514389,67.430359,9.849056,61.677814,...,30.721416,120.923409,64.141197,127.938820,11.455,45.789108,87.367889,92.663704,20.560509,21.780001


In [10]:
volumes = pd.DataFrame({t: data[t]['Volume'] for t in tickers_yahoo})
volumes.dropna(inplace=True)

dfs['volumes'] = volumes

print(volumes.shape)
volumes.head()

(505, 40)


,ADS.DE,AIR.PA,ALV.DE,BAS.DE,BAYN.DE,BEI.DE,BMW.DE,BNR.DE,CBK.DE,CON.DE,...,RWE.DE,SAP.DE,G24.DE,SIE.DE,ENR.DE,SHL.DE,SY1.DE,VOW3.DE,VNA.DE,ZAL.DE
Date,,,,,,,,,,,,,,,,,,,,,
2023-10-02,473416.0,769422,937778.0,2531880.0,1824295.0,202802.0,734361.0,228932.0,9709198.0,346772.0,...,2264868.0,1546211.0,283111.0,1035118.0,2660299.0,738909.0,204879.0,597856.0,2704758.0,1624392.0
2023-10-03,415385.0,834710,714829.0,1451608.0,2712839.0,320382.0,698279.0,192103.0,6253617.0,359542.0,...,4686266.0,1213513.0,198352.0,1119396.0,3257625.0,616025.0,178311.0,687105.0,2371561.0,1702983.0
2023-10-04,339492.0,647672,919410.0,2028974.0,1982779.0,400668.0,958144.0,464074.0,7778531.0,312442.0,...,3459742.0,1502474.0,132223.0,1209107.0,2767319.0,521336.0,316094.0,684218.0,3151904.0,1665908.0
2023-10-05,817310.0,589204,720576.0,1645567.0,1959463.0,202163.0,595090.0,262078.0,3984968.0,131863.0,...,1641405.0,1138484.0,98177.0,1130369.0,1993549.0,473991.0,233545.0,539346.0,1911472.0,1523226.0
2023-10-06,533804.0,766260,864116.0,1916315.0,2379227.0,391505.0,1008126.0,346728.0,5389082.0,321960.0,...,2679175.0,1559814.0,121808.0,0.0,4287041.0,917379.0,386180.0,593669.0,2918730.0,2447564.0


In [11]:
returns = close_prices.pct_change(fill_method=None) * 100
returns.dropna(inplace=True)

dfs['returns'] = returns

print(returns.shape)
returns.head()

(504, 40)


,ADS.DE,AIR.PA,ALV.DE,BAS.DE,BAYN.DE,BEI.DE,BMW.DE,BNR.DE,CBK.DE,CON.DE,...,RWE.DE,SAP.DE,G24.DE,SIE.DE,ENR.DE,SHL.DE,SY1.DE,VOW3.DE,VNA.DE,ZAL.DE
Date,,,,,,,,,,,,,,,,,,,,,
2023-10-03,-2.357233,-0.987578,-0.785815,-0.310345,-2.195486,-0.779009,-0.904637,-1.125143,-2.662867,-1.963122,...,-3.848422,-0.455803,-0.559176,-0.996721,-3.296255,-0.686498,-0.313120,-1.743126,-3.242772,-5.321617
2023-10-04,0.871088,-0.627412,-0.588365,0.215521,0.877390,-0.702477,0.104923,-1.470986,-0.830480,-0.728151,...,0.213878,-0.245301,1.405817,0.330572,-1.107798,1.424377,1.974418,0.429502,-0.317030,-1.319641
2023-10-05,-3.676288,-0.016190,0.478030,-1.206693,-0.892347,0.832290,-1.163518,0.281676,0.492611,-0.550125,...,0.731712,0.114757,-0.123233,-0.958519,-0.258507,-1.404373,0.462054,-0.985489,-0.272593,1.337288
2023-10-06,1.972330,0.615289,1.631180,1.148870,0.467293,1.155600,1.505989,1.348329,1.617639,1.874627,...,-0.938268,1.162590,1.727326,1.814615,-1.036716,-0.628400,-2.168210,0.262907,0.865593,6.451621
2023-10-10,6.669178,1.062116,0.423540,1.135821,1.554160,-0.489602,1.891129,0.166300,0.578883,0.060321,...,4.705160,0.485606,-0.454817,1.604054,4.626798,1.918208,2.798301,1.254922,2.348690,3.994485


## Macroeconomical data from Deutsche Bundesbank and Bundesagentur für Arbeit

In [12]:
def delete_last_comma(csv_file_existing, csv_file_cleaned):
    '''
    CSV files from Deutsche Bundesbank have one last comma
    per line, which needs to be removed in order to be read by Pandas.
    '''
    with open(f"source_data/{csv_file_existing}", "r") as f:
        lines = f.readlines()
    
    cleaned = [line.rstrip(",\n") + "\n" for line in lines]
    
    with open(f"data/{csv_file_cleaned}", "w") as f:
        f.writelines(cleaned)

In [13]:
delete_last_comma("BBEX3.D.USD.EUR.BB.AC.000.csv", "EURtoUSD.csv")
EURtoUSD = pd.read_csv("data/EURtoUSD.csv")
EURtoUSD = EURtoUSD[EURtoUSD['EUR 1 = USD'] != "No value available"]

dfs['EURtoUSD'] = EURtoUSD

EURtoUSD

,date,EUR 1 = USD
1,2023-10-02,1.0530
2,2023-10-03,1.0469
3,2023-10-04,1.0497
4,2023-10-05,1.0526
5,2023-10-06,1.0563
...,...,...
724,2025-09-24,1.1756
725,2025-09-25,1.1739
726,2025-09-26,1.1672
729,2025-09-29,1.1723


In [14]:
delete_last_comma("BBEX3.D.GBP.EUR.BB.AC.000.csv", "EURtoGBP.csv")
EURtoGBP = pd.read_csv("data/EURtoGBP.csv")
EURtoGBP = EURtoGBP[EURtoGBP['EUR 1 = GBP'] != "No value available"]

dfs['EURtoGBP'] = EURtoGBP

EURtoGBP

,date,EUR 1 = GBP
1,2023-10-02,0.86628
2,2023-10-03,0.86775
3,2023-10-04,0.86588
4,2023-10-05,0.86605
5,2023-10-06,0.86510
...,...,...
724,2025-09-24,0.87310
725,2025-09-25,0.87480
726,2025-09-26,0.87400
729,2025-09-29,0.87230


In [15]:
# European Central Bank Interest Rates
delete_last_comma("BBIN1.M.D0.ECB.ECBMIN.EUR.ME.csv", "interest_rates.csv")
interest_rates = pd.read_csv("data/interest_rates.csv")

dfs['interest_rates'] = interest_rates

interest_rates

,date,interest_rate
0,2023-10-31,4.50
1,2023-11-30,4.50
2,2023-12-31,4.50
3,2024-01-31,4.50
4,2024-02-29,4.50
5,2024-03-31,4.50
6,2024-04-30,4.50
7,2024-05-31,4.50
8,2024-06-30,4.25
9,2024-07-31,4.25


In [16]:
# Unemployment rates in Germany
unemployment = pd.read_csv("source_data/statistik_lzr_20251008213834.csv")
unemployment.drop(columns=["Unemployed population", "Access for the unemployed", "Outflow of unemployed"], inplace=True)
unemployment = unemployment[:24].copy()
unemployment["date"] = pd.to_datetime(unemployment["Reporting month"], format="%B %Y")
unemployment["date"] = unemployment["date"] + pd.offsets.MonthEnd(0)
unemployment["date"] = unemployment["date"].dt.strftime("%Y-%m-%d")
unemployment.drop(columns=["Reporting month"], inplace=True)
unemployment.sort_values(by=["date"], ignore_index=True, inplace=True)

dfs['unemployment'] = unemployment

unemployment

,Unemployment rate,date
0,5.7,2023-10-31
1,5.6,2023-11-30
2,5.7,2023-12-31
3,6.1,2024-01-31
4,6.1,2024-02-29
5,6.0,2024-03-31
6,6.0,2024-04-30
7,5.8,2024-05-31
8,5.8,2024-06-30
9,6.0,2024-07-31


## DAX monthly tables

In [17]:
# Using data from the last day of the month. 
# When the last day doesn't have data, use the previous date with data.

month_dates = list(unemployment['date'])
missing_dates = ["2023-12-29", 
                 "2024-03-28", "2024-06-28", "2024-08-30", "2024-11-29", "2024-12-30", 
                 "2025-05-30", "2025-08-29", "2025-09-29"]
for date in missing_dates:
    month_dates.append(date)

In [18]:
monthly_close_prices = close_prices[close_prices.index.isin(sorted(month_dates))].copy()
dfs['monthly_close_prices'] = monthly_close_prices

/var/folders/3j/f7d6p9t136x34vdmdy30wrrc0000gn/T/ipykernel_3639/3701614056.py:1: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  monthly_close_prices = close_prices[close_prices.index.isin(sorted(month_dates))].copy()


In [19]:
monthly_returns = monthly_close_prices.pct_change(fill_method=None) * 100
monthly_returns.dropna(inplace=True)
dfs['monthly_returns'] = monthly_returns

In [20]:
# Volumes sum up all volumes for each month
volumes.index = pd.to_datetime(volumes.index)
monthly_volumes = volumes.resample("M").sum()
monthly_volumes.index = monthly_volumes.index.strftime("%Y-%m-%d")
dfs['monthly_volumes'] = monthly_volumes

/var/folders/3j/f7d6p9t136x34vdmdy30wrrc0000gn/T/ipykernel_3639/300223943.py:3: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_volumes = volumes.resample("M").sum()


## Export data

In [26]:
## All dfs are stored inside this dictionary
dfs.keys()

dict_keys([])

In [24]:
for df_name, df in dfs.items():
    df.to_csv(f"data/{df_name}.csv", index=False)